# Create S1 RTC for Tindex (Super sites)

## Create S1 RTC stacks for each vector grid cell that intersects the Norway/Alaska super sites

- S1 RTC are in UTM
- Tindex is equal area above proj
- Datasets are transformed to 4326 for cross-referencing (as with HLS code)

In [1]:
# import libraries
import os
import rasterio as rio
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import folium
from pyproj import Transformer
from rio_tiler.io import COGReader
from rasterio.windows import from_bounds
import numpy as np
import numpy.ma as ma
from rasterio import enums
from rasterio.io import MemoryFile
from rasterio.crs import CRS
from rasterio.vrt import WarpedVRT
from rasterio.warp import array_bounds, calculate_default_transform
from rio_cogeo.profiles import cog_profiles
from rio_cogeo.cogeo import cog_translate

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# get vh and vv file lists
# create 4 lists of data based on season and pol
# eg, all S1 data that are spring + vh
root_dir = "/projects/my-public-bucket/sentinel1_seasonal_comps/"

vh_fall = []
vv_fall = []
vh_spring = []
vv_spring = []

for root, dirs, files in os.walk(root_dir, topdown=False):
    for name in files:
        if name.endswith('_median.tif'):
            if 'Spring' in name:
                if 'vv' in name:
                    vv_spring.append(os.path.join(root, name))
                elif 'vh' in name:
                    vh_spring.append(os.path.join(root, name))
            elif 'Fall' in name:
                if 'vv' in name:
                    vv_fall.append(os.path.join(root, name))
                elif 'vh' in name:
                    vh_fall.append(os.path.join(root, name))

#### Use these lists to create individual dfs where the geometry is the S1 image bbox

In [3]:
def get_season_pol_dicts(img_list):
# loop through image lists and get bbox of image
# return a df with image properties incl bbox for each image
    geom = []
    filename = []
    path = []
    season = []
    pol = []
    for img in img_list:
        with rio.open(img) as f:
            # get image buonds and transform to 4326
            xmin, ymin, xmax, ymax = f.bounds
            transformer = Transformer.from_crs(f.crs, "EPSG:4326")

            ul = transformer.transform(xmin,ymax)
            lr = transformer.transform(xmax,ymin)
            
            # get lists of attributes for the df
            
            #shapely box: minx, miny, maxx, maxy
            geom.append(box(ul[1],lr[0],lr[1],ul[0]))
            path.append(img)
            filename.append(img.split('/')[-1])
            season.append(img.split('/')[-1].split('_')[5])
            pol.append(img.split('/')[-1].split('_')[6])    
    
    # return a df
    return gpd.GeoDataFrame({'filename':filename, 'filepath':path, 'season':season, 'pol':pol, 'geometry':geom}, crs="EPSG:4326")
        

In [6]:
vv_spring_df = get_season_pol_dicts(vv_spring)
vh_spring_df = get_season_pol_dicts(vh_spring)
vv_fall_df = get_season_pol_dicts(vv_fall)
vh_fall_df = get_season_pol_dicts(vh_fall)

In [8]:
# print one of the dfs to see its content
# each row is an S1 RTC image with its properties and bbox
vv_spring_df.head()

,filename,filepath,season,pol,geometry
0,Alaska_fid_1_p94_f210_Spring_vv_median.tif,/projects/my-public-bucket/sentinel1_seasonal_...,Spring,vv,"POLYGON ((-144.15044 63.92430, -144.15044 65.9..."
1,Alaska_fid_2_p94_f205_Spring_vv_median.tif,/projects/my-public-bucket/sentinel1_seasonal_...,Spring,vv,"POLYGON ((-143.61164 62.43485, -143.61164 64.4..."
2,Alaska_fid_3_p131_f378_Spring_vv_median.tif,/projects/my-public-bucket/sentinel1_seasonal_...,Spring,vv,"POLYGON ((-147.54702 63.09330, -147.54702 65.1..."
3,Norway_fid_1_p37_f398_Spring_vv_median.tif,/projects/my-public-bucket/sentinel1_seasonal_...,Spring,vv,"POLYGON ((10.37714 57.10926, 10.37714 59.14395..."
4,Norway_fid_2_p15_f199_Spring_vv_median.tif,/projects/my-public-bucket/sentinel1_seasonal_...,Spring,vv,"POLYGON ((7.14069 60.49315, 7.14069 62.64137, ..."


#### Open the boreal tindex (modified to include a smaller number of tiles) and store some key properties

In [10]:
# open the borel tindex
tindex = '/projects/my-public-bucket/sentinel1_seasonal_comps/Tindex/boreal_tiles_v003_S1RTC.gpkg'
#tindex = '/projects/my-public-bucket/boreal_tiles_v003.gpkg'
tind = gpd.read_file(tindex)
# get output crs from tindex file
out_crs = tind.crs

In [11]:
# proj to 4326 for intersection
tind_gdf = tind.to_crs("EPSG:4326")

In [40]:
# check the locations of the two datasets
# BLUE: tindex tiles
# GREEN: S1 RTC bounds
m = folium.Map(location=[55, -50],zoom_start=2, tiles='OpenStreetMap')
style1 = {'fillColor': '#228B22', 'color': '#228B22'}
folium.GeoJson(vh_fall_df,style_function=lambda x:style1).add_to(m)
style2 = {'color': 'blue', 'weight':1}
folium.GeoJson(tind_gdf,style_function=lambda x:style2).add_to(m)
folium.LatLngPopup().add_to(m)
m

#### create a LUT where each row is a tindex tile, and columns include lists of S1 properties, such as filepaths for all images that intersect the tindex

In [41]:
def create_lut(df, tind_gdf):
# for each tile in borel_tindex check intersection with each
# set of season+pol image df (with bbox; all in 4326)
# return df of S1 filepaths that intersect tindex tile
# on a per season+pol basis
    tile_number = []
    s1_filepaths = []
    pols = []
    seasons = []
    geometry = []
    for i in range(0, len(tind_gdf), 1):
        geom = tind_gdf.iloc[i].geometry
        tile_id = tind_gdf.tile_num.iloc[i]
        sub_df = df[df.intersects(geom)==True]
        if len(sub_df) > 0:
            s1_filepaths.append(sub_df.filepath.values)
            tile_number.append(tile_id)
            pols.append(sub_df.pol.values)
            seasons.append(sub_df.season.values)
            geometry.append(geom)

    return gpd.GeoDataFrame({'tile_number':tile_number, 's1_filepath':s1_filepaths, 'pol':pols, 'season':seasons, 'geometry':geometry}, crs=out_crs)




In [42]:
vv_spring_lut = create_lut(vv_spring_df, tind_gdf)
vh_spring_lut = create_lut(vh_spring_df, tind_gdf)
vv_fall_lut = create_lut(vv_fall_df, tind_gdf)
vh_fall_lut = create_lut(vh_fall_df, tind_gdf)

In [44]:
# verify LUT is correct
# This tindex tile intersects 3 fall vv-pol images
vv_fall_lut[vv_fall_lut.tile_number==3553]

,tile_number,s1_filepath,pol,season,geometry
113,3553,[/projects/my-public-bucket/sentinel1_seasonal...,"[vv, vv, vv]","[Fall, Fall, Fall]","POLYGON ((-148.68403 64.15610, -147.02931 63.7..."


In [45]:
# CRS of df of S1 imges with a 4326 bbox 
vv_fall_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [46]:
# crs of LUT with S1 scenes that intersect the tindex
vv_fall_lut.crs

<Bound CRS: PROJCS["unnamed",GEOGCS["GRS 1980(IUGG, 1980)",DAT ...>
Name: unnamed
Axis Info [cartesian]:
- [east]: Easting (Meter)
- [north]: Northing (Meter)
Area of Use:
- undefined
Coordinate Operation:
- name: Transformation from GRS 1980(IUGG, 1980) to WGS84
- method: Position Vector transformation (geog2D domain)
Datum: unknown
- Ellipsoid: GRS80
- Prime Meridian: Greenwich
Source CRS: unnamed

,tile_number,s1_filepath,pol,season,geometry
113,3553,[/projects/my-public-bucket/sentinel1_seasonal...,"[vv, vv, vv]","[Fall, Fall, Fall]","POLYGON ((-148.68403 64.15610, -147.02931 63.7..."


In [37]:
# Function to write out the dtata to cog when finished
def write_cog(stack, out_fn: str, in_crs, src_transform, bandnames: list, out_crs=None, resolution: tuple=(30, 30), clip_geom=None, clip_crs=None, align:bool=False, input_nodata_value:int=None):
    '''
    Write a cloud optimized geotiff with compression from a numpy stack of bands with labels
    Reproject if needed, Clip to bounding box if needed.
    
    Parameters:
    stack: np.array 
        3d numpy array (bands, height, width) 
    out_fn: str
        Output Filename
    in_crs: str
        CRS of input raster
    src_transform: Affine
        Affine transform of imput raster
    bandnames: list[str]
        List of bandnames in band/dimension order that matches stack
    out_crs: CRS, optional
        If reprojecting, the output CRS
    clip_geom: dict, optional
        Polygon geometry as geojson dictionary
    clip_crs: CRS, optional
        CRS of clip_geom
    align: bool, optional
        True aligns the output raster with the top left corner of the clip_geom. clip_geom CRS must be the same as
        the out_crs.
    input_nodata_value: int, optional
        Setting to an int allows for reading in uint8 datatypes; otherwise assume np.nan
    '''
    
    #TODO: remove print statements, add debugging
    
    if out_crs is None:
        out_crs = in_crs
        
    if input_nodata_value is None:
        input_nodata_value = np.nan
   
    # Set the profile for the in memory raster based on the ndarry stack
    src_profile = dict(
        driver="GTiff",
        height=stack.shape[1],
        width=stack.shape[2],
        count=stack.shape[0],
        dtype=stack.dtype,
        crs=in_crs,
        transform=src_transform,
        nodata=input_nodata_value)

    # Set the reproject parameters for the WarpedVRT read
    vrt_params = {}
    if out_crs is not None:
        vrt_params["crs"] = out_crs
        vrt_params["src_crs"] = in_crs
        vrt_params["dtype"] = str(stack.dtype)
        vrt_params["nodata"] = input_nodata_value
        vrt_params["resampling"] = enums.Resampling.nearest # <---hard-coded nearest neighbor resampling prevents contamination from nan values and maintains numerical categories
        
        #TODO: Add  transform with resolution specification
        if out_crs != in_crs:
            left, bottom, right, top = array_bounds(height = src_profile["height"],
                   width = src_profile["width"], 
                   transform = src_profile["transform"])
            vrt_params["transform"], vrt_params["width"], vrt_params["height"] = calculate_default_transform(
                src_crs = in_crs,
                dst_crs = out_crs,
                left = left,
                bottom = bottom,
                right = right,
                top = top,
                width = src_profile["width"],
                height = src_profile["height"],
                resolution = resolution
                )
        if align is True:
            left, bottom, right, top = clip_geom.total_bounds
            vrt_params["transform"], vrt_params["width"], vrt_params["height"] = calculate_default_transform(
                src_crs = in_crs,
                dst_crs = out_crs,
                left = left,
                bottom = bottom,
                right = right,
                top = top,
                width = src_profile["width"],
                height = src_profile["height"],
                resolution = resolution
                )
        
    # Get the rio-cogeo profile for deflate compression, modify some of the options
    dst_profile = cog_profiles.get("deflate")
    dst_profile['blockxsize']=256
    dst_profile['blockysize']=256
    dst_profile['predictor']=1 # originally set to 2, which fails with 'int'; 1 tested successfully for 'int' and 'float64'
    dst_profile['zlevel']=7
    
    with MemoryFile() as memfile:
        with memfile.open(**src_profile) as mem:
            # Populate the input file with NumPy array
            # HERE; this memory file can be reprojected then saved
            mem.write(stack)

            if clip_geom is not None:
                # Do the clip to geometry (rasterio takes this; not in_bbox)
                # # https://rasterio.readthedocs.io/en/latest/topics/masking-by-shapefile.html
                clip_geom_json = clip_geom.__geo_interface__['features'][0]['geometry']
                vrt_params["cutline"] = create_cutline(mem, clip_geom_json, geometry_crs = clip_crs)
            
            for n in range(len(bandnames)):
                mem.set_band_description(n+1, bandnames[n])
        
            with WarpedVRT(mem,  **vrt_params) as vrt:
                cog_translate(
                    vrt,
                    # To avoid rewriting over the infile
                    out_fn,
                    dst_profile,
                    add_mask=True,
                    in_memory=True,
                    quiet=False)

    
    # TODO: return something useful
    return True


In [49]:
def read_and_merge_s1(lut, folder):
    # get each row of table in the lut
    for index, row in lut.iterrows():
        # Get list of files that overlap a tile
        paths = lut.s1_filepath.iloc[index]
        # get geometry of tile
        geom = lut.geometry.iloc[index]
        # Get tile number
        tile_num = str(lut.tile_number.iloc[index])
        # get pol
        pol = lut.pol.iloc[index][0]
        # get season
        season = lut.season.iloc[index][0]
        # set up list to hold arrs
        comp_list = []
        # set up out name for tile
        name = 'Sentinel1_rtc_' + tile_num + '_' + season + '_' + pol + '_mean.tif'
        # set up in_crs and tform info from first file in list (Warning: assumption that images that overlap a tile have same proj (or at least very close if UTM))
        with rio.open(paths[0]) as r:
            in_crs = r.crs
            in_tform = r.transform
        # iterate over the s1 files that intersect tile
        for file in paths:
            # window read the S1
            with COGReader(file) as cog:
                # get img bunds
                in_bbox = geom.bounds
                # set crs
                bounds_crs = "EPSG:4326"
                dst_crs = "EPSG:4326"
                # set image size
                height = 3000
                width = 3000
                # Do window read
                img = cog.part(in_bbox, bounds_crs=bounds_crs, max_size=None, dst_crs=dst_crs, height=height, width=width)
                # append data to list where ther is some
                # skips empty tiles where there is no S1 (or just nans)
                if np.amax(img.data) == 0:
                    pass
                else:
                    # append the data to a list
                    comp_list.append(np.where(img.data==0, np.nan, img.data))
        
        # merge lists to 1 array and reduce to mean
        if len(comp_list) > 0:
            comp_arrays = np.array(comp_list)
            mean_arr = np.nanmean(comp_arrays,axis=0)
            
            # write S1 to cog
            write_cog(mean_arr, 
              os.path.join('/projects/my-public-bucket/sentinel1_seasonal_comps/S1_RTC_tindex/' + folder, name), 
              in_crs, 
              in_tform, 
              [pol], 
              out_crs=out_crs, 
              resolution=(30, 30), 
              align=False
              #clip_geom=tile_id["geom_orig"] # this was added late to address some HLS output showing 2999 rows..now this matches how topo stacks are built. Does not correct issue.
             )
             

In [50]:
# make the S1 images for season + pol of interest
read_and_merge_s1(vv_spring_lut, 'Spring_vv')
#read_and_merge_s1(vh_spring_lut, 'Spring_vh')
#read_and_merge_s1(vh_fall_lut, 'Fall_vh')
#read_and_merge_s1(vv_fall_lut, 'Fall_vv')